<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 824ms/step - loss: 0.6714 - accuracy: 0.5572 - val_loss: 0.7100 - val_accuracy: 0.4439
Epoch 2/10
26/26 [==============================] - 21s 792ms/step - loss: 0.6241 - accuracy: 0.6679 - val_loss: 0.6060 - val_accuracy: 0.6390
Epoch 3/10
26/26 [==============================] - 21s 800ms/step - loss: 0.5910 - accuracy: 0.7032 - val_loss: 0.6275 - val_accuracy: 0.5902
Epoch 4/10
26/26 [==============================] - 20s 788ms/step - loss: 0.5700 - accuracy: 0.7092 - val_loss: 0.5211 - val_accuracy: 0.7805
Epoch 5/10
26/26 [==============================] - 20s 762ms/step - loss: 0.4894 - accuracy: 0.7798 - val_loss: 0.5282 - val_accuracy: 0.6878
Epoch 6/10
26/26 [==============================] - 20s 765ms/step - loss: 0.4687 - accuracy: 0.7810 - val_loss: 0.5026 - val_accuracy: 0.7268
Epoch 7/10
26/26 [==============================] - 21s 796ms/step - loss: 0.4063 - accuracy: 0.8212 - val_loss: 0.3572 - val_accuracy: 0.8878

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.66.h5
26/26 - 22s - loss: 0.6776 - accuracy: 0.5973 - val_loss: 0.6606 - val_accuracy: 0.6098
Epoch 2/5

Epoch 00002: saving model to weights.02-0.69.h5
26/26 - 25s - loss: 0.6246 - accuracy: 0.6825 - val_loss: 0.6925 - val_accuracy: 0.5073
Epoch 3/5

Epoch 00003: saving model to weights.03-0.54.h5
26/26 - 24s - loss: 0.5776 - accuracy: 0.7153 - val_loss: 0.5413 - val_accuracy: 0.7561
Epoch 4/5

Epoch 00004: saving model to weights.04-0.48.h5
26/26 - 23s - loss: 0.5192 - accuracy: 0.7616 - val_loss: 0.4783 - val_accuracy: 0.8195
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 23s - loss: 0.4593 - accuracy: 0.7895 - val_loss: 0.4088 - val_accuracy: 0.8780


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 28s - loss: 0.6660 - accuracy: 0.5572 - val_loss: 0.6541 - val_accuracy: 0.5415


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 23s - loss: 0.6707 - accuracy: 0.5547 - val_loss: 0.6538 - val_accuracy: 0.5415
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 22s - loss: 0.6002 - accuracy: 0.7068 - val_loss: 0.5826 - val_accuracy: 0.6390


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 22s - loss: 0.6772 - accuracy: 0.5949 - val_loss: 0.8581 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 23s - loss: 0.6352 - accuracy: 0.6460 - val_loss: 0.5806 - val_accuracy: 0.7756
Epoch 3/50
26/26 - 22s - loss: 0.5881 - accuracy: 0.6971 - val_loss: 0.5487 - val_accuracy: 0.7756
Epoch 4/50
26/26 - 22s - loss: 0.5076 - accuracy: 0.7810 - val_loss: 0.5585 - val_accuracy: 0.6878
Epoch 5/50
26/26 - 22s - loss: 0.4454 - accuracy: 0.8017 - val_loss: 0.4244 - val_accuracy: 0.8390
Epoch 6/50
26/26 - 22s - loss: 0.3910 - accuracy: 0.8443 - val_loss: 0.4089 - val_accuracy: 0.8439
Epoch 7/50
26/26 - 23s - loss: 0.3082 - accuracy: 0.8966 - val_loss: 0.2621 - val_accuracy: 0.9220
Epoch 8/50
26/26 - 21s - loss: 0.2788 - accuracy: 0.8954 - val_loss: 0.2049 - val_accuracy: 0.9561
Epoch 9/50
26/26 - 20s - loss: 0.2071 - accuracy: 0.9428 - val_loss: 0.1762 - val_accuracy: 0.9610
Epoch 10/50
26/26 - 20s - loss: 0.1577 - accuracy: 0.9562 - val_loss: 0.2664 - val_accuracy: 0.8780
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 22s 844ms/step - loss: 0.6744 - accuracy: 0.5645 - val_loss: 0.6677 - val_accuracy: 0.5805
Epoch 2/5
26/26 [==============================] - 20s 766ms/step - loss: 0.6506 - accuracy: 0.6423 - val_loss: 0.6422 - val_accuracy: 0.7171
Epoch 3/5
26/26 [==============================] - 20s 769ms/step - loss: 0.6223 - accuracy: 0.6691 - val_loss: 0.6333 - val_accuracy: 0.5854
Epoch 4/5
26/26 [==============================] - 20s 758ms/step - loss: 0.5774 - accuracy: 0.7226 - val_loss: 0.5409 - val_accuracy: 0.8000
Epoch 5/5
26/26 [==============================] - 20s 765ms/step - loss: 0.5303 - accuracy: 0.7603 - val_loss: 0.4840 - val_accuracy: 0.7756


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.564477,0.674558,0.580488,0.667679
1,1,0.642336,0.650878,0.717073,0.642204
2,2,0.669100,0.623082,0.585366,0.633252
3,3,0.722628,0.576887,0.800000,0.540894
4,4,0.760341,0.530783,0.775610,0.483999


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 31s 1s/step - loss: 0.6864 - accuracy: 0.5377 - val_loss: 0.6929 - val_accuracy: 0.4732

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6627 - accuracy: 0.5925 - val_loss: 0.6781 - val_accuracy: 0.5073

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 776ms/step - loss: 0.6533 - accuracy: 0.6071 - val_loss: 0.6685 - val_accuracy: 0.5366

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6479 - accuracy: 0.6217 - val_loss: 0.6638 - val_accuracy: 0.5512

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 769ms/step - loss: 0.6448

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

In [ ]:
%tensorboard --logdir log_dir